In [1]:
import pprint
import pickle
import string
from tabulate import tabulate
import pandas as pd

import sys
import os
import time

import numpy as np
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
import matplotlib
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn.externals import joblib

In [2]:
full_program_time_1 = time.time()
for num in range(3, 4):
    
    #Load the file from pickle
    print('Load : File ', num)
    
    #Read the train data
    pkl_file = open('outputs/pkl/preprocessed_data_%s.pkl'%num, 'rb')
    data = pickle.load(pkl_file)
    pkl_file.close()
    
    #Read the test data which is real data
    pkl_file = open('outputs/pkl/preprocessed_test_data_1.pkl', 'rb')
    testdata = pickle.load(pkl_file)
    pkl_file.close()

    if num >= 6:
        
        #Untokenize the data
        print('Untokenize : File ', num)
        
        count = 0
        for line in data.content:
            data.content[count] = "".join([" "+ a if not a.startswith("'") and a not in string.punctuation else a for a in line]).strip()
            count = count + 1
    
    X_train = data.content
    y_train = data.sentiment
    
    
    X_test = testdata.content
    y_test = testdata.sentiment
    
    
    # Binarize the output
    y_train = label_binarize(y_train, classes=['neutral', 'surprise', 'empty', 'happiness', 'love', 'fun', 'relief', 'enthusiasm', 'worry', 'sadness', 'hate', 'boredom', 'anger'])
    y_test = label_binarize(y_test, classes=['neutral', 'surprise', 'empty', 'happiness', 'love', 'fun', 'relief', 'enthusiasm', 'worry', 'sadness', 'hate', 'boredom', 'anger'])
    n_classes = y_train.shape[1]
    
    vectorizer = TfidfVectorizer(min_df = 5,
                                max_df = 0.8,
                                sublinear_tf = True,
                                use_idf = True)

    
#     #Split the data into 7:3 ratio
#     X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state = 123)

    
    #Transform the words to feature vectors
    
    print('Transform : File ', num)
    
    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)
    
    
    ########## SVM ##########
    
    print('Training : File ', num)
    
    classifier = OneVsRestClassifier(svm.SVC(kernel='linear', probability=True))
    
    y_score = classifier.fit(X_train, y_train).decision_function(X_test)
    
    ########## SVM ##########
    
    
    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    
    
    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
    
    
    # Compute macro-average ROC curve and ROC area
    # First aggregate all false positive rates
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

    # Then interpolate all ROC curves at this points
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(n_classes):
        mean_tpr += interp(all_fpr, fpr[i], tpr[i])

    # Finally average it and compute AUC
    mean_tpr /= n_classes

    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])
    
    
    lw = 2
    fig = plt.figure()
    plt.plot(fpr["micro"], tpr["micro"],
             label='micro-average ROC curve (area = {0:0.2f})'
                   ''.format(roc_auc["micro"]),
             color='deeppink', linestyle=':', linewidth=4)

    plt.plot(fpr["macro"], tpr["macro"],
             label='macro-average ROC curve (area = {0:0.2f})'
                   ''.format(roc_auc["macro"]),
             color='navy', linestyle=':', linewidth=4)

    colors = cycle(['greenyellow', 'limegreen', 'lime', 'darkblue', 'slateblue', 'dodgerblue', 'skyblue', 'cadetblue', 'firebrick', 'red', 'darksalmon', 'sienna', 'sandybrown'])
    for i, color in zip(range(n_classes), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=lw,
                 label='ROC curve of class {0} (area = {1:0.2f})'
                 ''.format(i, roc_auc[i]))
    

    plt.plot([0,1], [0, 1], 'k--', lw=lw)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Some extension of Receiver operating characteristic to multi-class')
    lgd = plt.legend(bbox_to_anchor=(1.05, 1), loc=2)
    
    
    print('Save : File ', num)
    
    result = [['svm_linear_result', y_score, time_1, time_2]]
        
    df = pd.DataFrame(result, columns=['Type of kernel (SVM)','Y Score', 'Time_1', 'Time_2'])    
    
    #Persist the model & Vectorizer
    joblib.dump(classifier, 'Backup/SVM/2.2_svm_model_roc_%s_3.pkl'%num)
    joblib.dump(vectorizer, 'Backup/SVM/2.2_svm_model_roc_%s_vectorizer_3.pkl'%num)
    
    fig.savefig('outputs/graph/SVM/roc_preprocessed_data_result_%s_3.png'%num, bbox_extra_artists = (lgd,), bbox_inches = 'tight')
    
    outputs = open('Outputs/pkl/SVM/2.2_svm_roc_%s_result_3.pkl'%num,'wb')
    pickle.dump(df, outputs)
    outputs.close()
    
    print('File ' ,num , ' is completed.\n')

    
full_program_time_2 = time.time()
second = full_program_time_2 - full_program_time_1
minute = second/60
hour = minute/60
print('Minute :', minute)
print('Hour :', hour)

Load : File  3
Transform : File  3
Training : File  3
Save : File  3
File  3  is completed.

Minute : 161.1684769709905
Hour : 2.6861412828498414
